In [2]:
import json
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras import Input, Model, models, layers

In [3]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("output/Combined_dataset.csv")
df.head(1)

,Unnamed: 0,backers_count,blurb,category,converted_pledged_amount,country,created_at,current_currency,deadline,fx_rate,goal,id,launched_at,location,name,pledged,profile,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,story,faq,num_faq,comments,n_comments,duration
0,0,1,With your help we will create this device that...,"{""id"":331,""name"":""3D Printing"",""slug"":""technol...",1,ES,2015-08-18 21:01,USD,2016-07-09 20:11,1.212886,15000,1662798399,2016-06-09 20:11,"{""id"":55897673,""name"":""Mexico"",""slug"":""mexico-...",Save water 100% liquid downloads in toilets at...,1.0,"{""id"":2075449,""project_id"":2075449,""state"":""in...",save-water-100-liquid-downloads-in-toilets,https://www.kickstarter.com/discover/categorie...,False,False,failed,2016-07-09 20:11,1.13693,"{""web"":{""project"":""https://www.kickstarter.com...",1.13693,domestic,Producto Final\nHi! The funds we will used to ...,[' '],0,Only backers can post comments. Log in\nNo com...,0,30 days 00:00:00.000000000


In [29]:
def check_location(data,column):
    try:
        return (json.loads(data)[column])
    except:
        return "nan"

def clean_df(df):
    category = list(map(lambda x: json.loads(x)["slug"], df["category"]))
    df["category"] = [x[0] for x in category]
    df["sub_category"] = [x[1] if len(x) == 2 else "" for x in category]
    df["type"] = list(map(lambda x: check_location(x,"type"), df["location"]))
    df["country"] = list(map(lambda x: check_location(x,"expanded_country"), df["location"]))
    return df

def tfidf_vec(corpus):
    vectorizer = TfidfVectorizer()
    result = vectorizer.fit_transform(corpus)
    return [len(vectorizer.vocabulary_),result]

In [31]:
# df_ = clean_df(df)
# blurb_shape, df["blurb_tfidf"] = tfidf_vec(df["blurb"])
df_.head(1)

,Unnamed: 0,backers_count,blurb,category,converted_pledged_amount,country,created_at,current_currency,deadline,fx_rate,goal,id,launched_at,location,name,pledged,profile,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,story,faq,num_faq,comments,n_comments,duration,sub_category,type,blurb_tfidf
0,0,1,With your help we will create this device that...,t,1,Mexico,2015-08-18 21:01,USD,2016-07-09 20:11,1.212886,15000,1662798399,2016-06-09 20:11,"{""id"":55897673,""name"":""Mexico"",""slug"":""mexico-...",Save water 100% liquid downloads in toilets at...,1.0,"{""id"":2075449,""project_id"":2075449,""state"":""in...",save-water-100-liquid-downloads-in-toilets,https://www.kickstarter.com/discover/categorie...,False,False,failed,2016-07-09 20:11,1.13693,"{""web"":{""project"":""https://www.kickstarter.com...",1.13693,domestic,Producto Final\nHi! The funds we will used to ...,[' '],0,Only backers can post comments. Log in\nNo com...,0,30 days 00:00:00.000000000,,Suburb,"(0, 215)\t0.3206657811635917\n (0, 2)\t0.22..."


In [33]:
nlp_in = Input(shape = (1000,))
# emb = Embedding(output_dim=32, input_dim=blurb_shape, input_length=1000)(nlp_input) 
# nlp_out = Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=regularizers.l2(0.01)))(emb)

In [15]:
nlp_input = Input(shape=(seq_length,), name='nlp_input')

NameError: name 'vectorizer' is not defined

In [16]:
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(df["blurb"])

<1000x5181 sparse matrix of type '<class 'numpy.float64'>'
	with 15981 stored elements in Compressed Sparse Row format>

In [2]:
import nltk
import string
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

max_len = 17000

In [3]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("output/Combined_dataset.csv")
df_ = df[["story","state"]].dropna().reset_index(inplace = False)[["story","state"]]

In [4]:
words = set(nltk.corpus.words.words())
def remove_non_english(string):
    return " ".join(word.lower() for word in nltk.wordpunct_tokenize(string) \
                   if word.lower() in words or not word.isalpha()) 
def remove_puct(text):
    table = str.maketrans("","",string.punctuation)
    return text.translate(table)


In [5]:
df_['story_en_'] = list(map(lambda x: remove_non_english(x),df_["story"]))
df_['story_en'] = list(map(lambda x: remove_puct(x),df_["story_en_"]))
df_["state"] = df.state.replace(to_replace=['failed', 'successful'], value=[0, 1])
# max([len(i) for i in df_["story_en"]])

In [6]:
def create_corpus(df):
    corpus = []
    for text in df["story_en"]:
        words = [word for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

corpus = create_corpus(df_)

In [7]:
num_words = len(corpus)

In [8]:
train_index = np.random.choice(len(df_),int(len(df_)*0.8),replace = False)

In [9]:
X_train = df_["story_en"][train_index]
y_train = df_["state"][train_index]

X_test = df_["story_en"][~df_.index.isin(train_index)]
y_test = df_["state"][~df_.index.isin(train_index)]


In [10]:
tokenizer = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(X_train)

In [11]:
X_train_seq = tokenizer.texts_to_sequences(X_train)

In [12]:
X_train_padded = pad_sequences(
    X_train_seq, maxlen = max_len, truncating = "post", padding = "post"
)

In [13]:
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(
    X_test_seq, maxlen = max_len, truncating = "post", padding = "post"
)

In [14]:
embedding_dict = {}
with open("Raw Data/glove.6B.100d.txt", encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:],"float32")
        embedding_dict[word] = vectors
f.close()

In [15]:
word_index = tokenizer.word_index

In [16]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words,100))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [17]:
model = Sequential()

model.add(
    Embedding(
    num_words,
    100,
    embeddings_initializer = Constant(embedding_matrix),
        input_length = max_len,
        trainable = False,
    )
)
model.add(LSTM(100, dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))
optimizer = Adam(learning_rate = 5e-3)

model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

In [ ]:
history = model.fit(
    X_train_padded,
    y_train,
    epochs = 10,
    validation_data = (X_test_padded, y_test),
    verbose = 1
)

Epoch 1/10
25/25 [==============================] - 3357s 134s/step - loss: 0.6628 - accuracy: 0.6256 - val_loss: 0.6523 - val_accuracy: 0.6548
Epoch 2/10
25/25 [==============================] - 3607s 144s/step - loss: 0.6635 - accuracy: 0.6256 - val_loss: 0.6449 - val_accuracy: 0.6548
Epoch 3/10
25/25 [==============================] - 4567s 183s/step - loss: 0.6635 - accuracy: 0.6256 - val_loss: 0.6514 - val_accuracy: 0.6548
Epoch 4/10
25/25 [==============================] - 4182s 167s/step - loss: 0.6641 - accuracy: 0.6256 - val_loss: 0.6515 - val_accuracy: 0.6548
Epoch 5/10
25/25 [==============================] - 4139s 166s/step - loss: 0.6630 - accuracy: 0.6256 - val_loss: 0.6476 - val_accuracy: 0.6548
Epoch 6/10
25/25 [==============================] - 3563s 143s/step - loss: 0.6617 - accuracy: 0.6256 - val_loss: 0.6457 - val_accuracy: 0.6548
Epoch 7/10
19/25 [=====================>........] - ETA: 34:12 - loss: 0.6599 - accuracy: 0.6316